In [1]:
import pandas as pd 

from showupforhealth.params import *
from showupforhealth.interface import *
from showupforhealth.ml_functions.preprocessor import *

# Importing default Libraries
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
import seaborn as sns
import warnings

# Hi-resolution Plots and Matplotlib inline
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

# Set the maximum number of rows and columns to be displayed
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000
warnings.filterwarnings('ignore')

# "magic commands" to enable autoreload of your imported packages
%load_ext autoreload
%autoreload 2

In [2]:
data = pd.read_csv(f'{OUTPUT_DATA}training_data_no_features.csv')


In [3]:
data.head(2)

,Appointment booked date,Appointment date,Appointment status,Appointment time,Booked by,Clinician,Rota type,Patient ID,app datetime,temp,precipitation,Age in years,Postcode,Sex,Registration date,Ethnicity category,Language,Registration status,FRAILTY,DEPRESSION,OBESITY,IHD,DM,HPT,NDHG,SMI,Latitude,Longitude,IMD2023,dist_to_station,distance_from_surg
0,10-May-22,14-Jul-22,Finished,10:00 - 10:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Registrar,19581842,2022-07-14 10:00:00,21.6,0.0,73,SW5 9BY,Male,2009-08-17 00:00:00,Other Mixed,(XaG5t) Main spoken language English,"Deceased, Deducted",0.0,0,0,0,0,0,0,0,51.490204,-0.193625,16808.0,0.136631,0.204712
1,10-May-22,14-Jul-22,Finished,10:00 - 10:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Registrar,19581842,2022-07-14 10:00:00,21.6,0.0,73,SW5 9BY,Male,2009-08-17 00:00:00,Other,(XaG5t) Main spoken language English,"Deceased, Deducted",0.0,0,0,0,0,0,0,0,51.490204,-0.193625,16808.0,0.136631,0.204712


In [12]:

def feature_engeneering(df):
    print('- Feature Engineering......')
    
    booked_by(df)
    
    encode_appointment_status(df)
    
    hash_patient_id(df)
    
    group_ethnicity_categories(df)
    
    format_datetime_columms(df)
    
    months_registered(df)
    
    map_rota_type(df)
    
    labelencode_sex(df)
    
    filter_current_registration(df)
    
    calculate_days_difference(df)
    
    ohe_ethnicity(df)
    
    return df

In [13]:
%%time
feature_engeneering(data)

- Feature Engineering......
📙 Booked by Clinician
🔠 Encode Appointment status
🚯 Hash Patient ID
🤦🏽‍♂️ Mapping Ethnicity Category
🕣 Convert Datetime Columns
📅 Months Registered with practice
🗺️ Map Rota types
🏷️ Labelencoding Sex
💧 Deseased and deducted
📅 Months Registered with practice
🔥 OHE Ethnicity
CPU times: user 10 s, sys: 488 ms, total: 10.5 s
Wall time: 10.5 s


,Appointment booked date,Appointment date,Appointment status,Appointment time,Booked by,Clinician,Rota type,Patient ID,app datetime,temp,precipitation,Age in years,Postcode,Sex,Registration date,Ethnicity category,Language,Registration status,FRAILTY,DEPRESSION,OBESITY,IHD,DM,HPT,NDHG,SMI,Latitude,Longitude,IMD2023,dist_to_station,distance_from_surg,booked_by_clinician,App_status_encoded,months_registered,Days Difference
0,2022-05-10,2022-07-14,Finished,10:00 - 10:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",GP,acd5dd39,2022-07-14 10:00:00,21.6,0.0,73,SW5 9BY,2,2009-08-17,NaN,(XaG5t) Main spoken language English,"Deceased, Deducted",0.00,0,0,0,0,0,0,0,51.490204,-0.193625,16808.0,0.136631,0.204712,1,1,155.0,65
1,2022-05-10,2022-07-14,Finished,10:00 - 10:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",GP,acd5dd39,2022-07-14 10:00:00,21.6,0.0,73,SW5 9BY,2,2009-08-17,NaN,(XaG5t) Main spoken language English,"Deceased, Deducted",0.00,0,0,0,0,0,0,0,51.490204,-0.193625,16808.0,0.136631,0.204712,1,1,155.0,65
2,2022-05-10,2022-07-14,Finished,10:00 - 10:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",GP,acd5dd39,2022-07-14 10:00:00,21.6,0.0,73,SW5 9BY,2,2009-08-17,NaN,(XaG5t) Main spoken language English,"Deceased, Deducted",0.00,0,0,0,0,0,0,0,51.490204,-0.193625,16808.0,0.136631,0.204712,1,1,155.0,65
3,2022-05-12,2022-07-05,Finished,09:00 - 09:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Nurse,4031b233,2022-07-05 09:00:00,17.7,0.0,40,W8 6JQ,0,2014-01-07,NaN,(XaG62) Main spoken language Portuguese,Current,0.00,0,0,0,0,0,0,0,51.496001,-0.193468,22698.0,0.519419,0.816996,1,1,102.0,54
4,2022-05-17,2022-07-05,Finished,10:00 - 10:59,"LE TELLIER, Jean-Pierre","FLORKO, Aleksandra (Miss)",Nurse,728a6013,2022-07-05 10:00:00,18.5,0.0,70,SW3 3SQ,2,2017-03-06,NaN,(XaG5t) Main spoken language English,Current,0.39,0,1,0,1,1,0,0,51.490771,-0.168178,10169.0,0.540792,1.656309,0,1,64.0,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1026752,2019-12-31,2019-12-31,Finished,15:00 - 15:59,"AMADI, Kasi (Miss)","PATEL, Rinku (Pharmacist)",ARRS,7ab7de9c,2019-12-31 15:00:00,8.9,0.0,87,SW10 9NG,2,2003-09-09,NaN,(XaG5t) Main spoken language English,"Deceased, Deducted",0.00,0,0,0,0,0,0,0,51.484323,-0.180246,20040.0,1.056940,0.579572,0,1,196.0,0
1026753,2019-12-31,2019-12-31,Finished,15:00 - 15:59,"AMADI, Kasi (Miss)","PATEL, Rinku (Pharmacist)",ARRS,9df8dccc,2019-12-31 15:00:00,8.9,0.0,36,SW5 0DY,2,2012-05-02,NaN,(XaG5t) Main spoken language English,Current,0.06,0,0,0,1,0,0,0,51.489257,-0.189864,14414.0,0.386273,1.348580,0,1,92.0,0
1026754,2019-12-31,2019-12-31,Finished,15:00 - 15:59,"AMADI, Kasi (Miss)","PATEL, Rinku (Pharmacist)",ARRS,d53300af,2019-12-31 15:00:00,8.9,0.0,42,SW10 9AL,0,2011-02-10,NaN,(XaG5t) Main spoken language English,Current,0.08,0,0,0,0,0,0,0,51.488467,-0.191806,14414.0,0.287596,1.445016,0,1,107.0,0
1026755,2019-12-31,2019-12-31,Finished,15:00 - 15:59,"DUFFIELD, Katie (Miss)","PATEL, Rinku (Pharmacist)",ARRS,8e2ae910,2019-12-31 15:00:00,8.9,0.0,51,SW5 9BY,0,1976-11-26,NaN,(XaG5t) Main spoken language English,Current,0.06,1,1,0,0,0,0,0,51.490204,-0.193625,16808.0,0.136631,1.629449,0,1,518.0,0
